In [ ]:
# prompt: mount my drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import shutil
import concurrent.futures
from datasets import load_dataset
import yt_dlp

def get_video_duration(ytid) -> float:
    """
    Retrieve the total duration (in seconds) of a YouTube video using yt-dlp with skip_download=True.
    Returns None if any error occurs or no duration is found.
    """
    url = f"https://www.youtube.com/watch?v={ytid}"
    info_opts = {
        "quiet": True,
        "skip_download": True,
    }
    try:
        with yt_dlp.YoutubeDL(info_opts) as ydl:
            info = ydl.extract_info(url, download=False)
        if "duration" in info:
            return info["duration"]
    except Exception as e:
        print(f"Could not retrieve metadata for {ytid}: {e}")
    return None

def download_youtube_audio_snippet_wav(ytid, start_time, snippet_length=10, output_dir="downloads"):
    """
    Downloads a snippet (from start_time to start_time+snippet_length or the end of the video)
    and converts it to WAV using ffmpeg. Returns the local file path if successful.
    """
    duration = get_video_duration(ytid)
    if duration is None:
        print(f"No duration found for {ytid}; skipping.")
        return None

    end_time = min(start_time + snippet_length, duration)
    if end_time <= start_time:
        print(f"Snippet start_time ({start_time}s) >= video duration ({duration}s) for {ytid}, skipping.")
        return None

    url = f"https://www.youtube.com/watch?v={ytid}"
    os.makedirs(output_dir, exist_ok=True)

    outtmpl = os.path.join(output_dir, f"{ytid}.%(ext)s")
    ydl_opts = {
        "format": "bestaudio/best",
        "quiet": True,
        "outtmpl": outtmpl,
        "external_downloader": "ffmpeg",
        "external_downloader_args": [
            "-ss", str(start_time),
            "-t", str(end_time - start_time)
        ],
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "wav",
                "preferredquality": "192",
            }
        ],
    }

    downloaded_file = None
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(url, download=True)
            if "requested_downloads" in info:
                downloaded_file = info["requested_downloads"][0]["filepath"]
            else:
                downloaded_file = os.path.join(output_dir, f"{ytid}.wav")
        except Exception as e:
            print(f"Error downloading snippet for {ytid}: {e}")
            return None

    if not downloaded_file or not os.path.exists(downloaded_file):
        print(f"Snippet file not found for {ytid}.")
        return None

    return downloaded_file

def process_example(example, split, drive_folder, output_dir="downloads"):
    """
    Process a single example:
      - Extract youtube_id and start_time.
      - Skip if the file already exists in Google Drive.
      - Download the audio snippet.
      - Move the resulting WAV file to Google Drive.
    """
    ytid = example["ytid"]
    if "start_s" not in example or example["start_s"] is None:
        print(f"No start_time for {ytid}, skipping.")
        return

    start_time = float(example["start_s"])
    drive_split_folder = os.path.join(drive_folder, "audio_two", split)
    os.makedirs(drive_split_folder, exist_ok=True)
    drive_wav_path = os.path.join(drive_split_folder, f"{ytid}.wav")

    # Check if the file already exists
    if os.path.exists(drive_wav_path):
        print(f"Already exists in Google Drive, skipping: {drive_wav_path}")
        return

    # Proceed with downloading
    local_wav = download_youtube_audio_snippet_wav(
        ytid,
        start_time=start_time,
        snippet_length=10,
        output_dir=output_dir
    )
    if not local_wav or not os.path.exists(local_wav):
        print(f"Failed to download snippet for {ytid}")
        return

    try:
        shutil.move(local_wav, drive_wav_path)
        print(f"Moved {local_wav} to Google Drive: {drive_wav_path}")
    except Exception as e:
        print(f"Error moving {local_wav} to Google Drive: {e}")


def process_dataset(ds, split, drive_folder, max_workers=8):
    """
    Process an entire dataset split using multi-threading.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_example, example, split, drive_folder) for example in ds]
        for f in concurrent.futures.as_completed(futures):
            try:
                f.result()
            except Exception as e:
                print(f"Exception in thread: {e}")

def main():
    ds = load_dataset("google/MusicCaps", split="train").shuffle(seed=42)
    total_files = len(ds)
    val_size = int(0.2 * total_files)
    train_ds = ds.select(range(val_size, total_files))
    val_ds = ds.select(range(val_size))
    drive_folder = "/content/drive/MyDrive"

    print(f"Total training files: {len(train_ds)}")
    print(f"Total validation files: {len(val_ds)}")

    process_dataset(train_ds, "train", drive_folder)
    process_dataset(val_ds, "val", drive_folder)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.06k [00:00<?, ?B/s]

musiccaps-public.csv:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5521 [00:00<?, ? examples/s]

Total training files: 4417
Total validation files: 1104
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/emP2Sg9QEg4.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/4ezo771lGts.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/5XNalwqtkFg.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/QBhhtVMiQBQ.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/AeDNy6Ff7VE.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/L0-l1LIa22g.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/KiFQFxJphjI.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/OWbfAmrFdp8.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_two/train/2T1P9ovsl4A.wav
Already exists in Google Drive, skipping: /content/drive/MyDrive/audio_

...

Could not retrieve metadata for AaUZb-iRStE: ERROR: [youtube] AaUZb-iRStE: Video unavailable
No duration found for AaUZb-iRStE; skipping.
Failed to download snippet for AaUZb-iRStE


ERROR: [youtube] FtskdD6Py7Y: Video unavailable


Could not retrieve metadata for FtskdD6Py7Y: ERROR: [youtube] FtskdD6Py7Y: Video unavailable
No duration found for FtskdD6Py7Y; skipping.
Failed to download snippet for FtskdD6Py7Y




ERROR: ffmpeg exited with code 1


Error downloading snippet for -cQ-jUTEgck: ERROR: ffmpeg exited with code 1
Failed to download snippet for -cQ-jUTEgck


In [ ]:
!pip install datasets yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
